In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import MFT
from checklist.inv_dir import INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb

In [2]:
editor = checklist.editor.Editor()
editor.tg
nlp = spacy.load('en_core_web_sm')

In [3]:
import sys
import os
import pandas as pd
pd.set_option('display.max_colwidth', 180)

mltest_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml-tests"))
sys.path.append(mltest_path)
mlfeedback_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml_feedback"))
sys.path.append(mlfeedback_path)

from ml_feedback.analysis.qqp_utils import *


# get the model
model_names = {
    "qqp": "QQP-all",
}
from ml_feedback.analysis.qqp_utils import *
models = load_models_by_name(model_names)

from checklist.pred_wrapper import PredictorWrapper
model = models["qqp"]
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)

/home/wtshuang/datasets/models/QQP-all
Namespace(adam_epsilon=1e-08, cache_dir='', config_name='', data_dir='/home/wtshuang/datasets/raw/glue_data//QQP/all', device=device(type='cuda'), do_eval=True, do_lower_case=True, do_train=True, eval_all_checkpoints=False, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, label_list='None', learning_rate=2e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_seq_length=512, max_steps=-1, model_name_or_path='bert-base-uncased', model_type='bert', n_gpu=2, no_cuda=False, num_train_epochs=3.0, output_dir='//home/wtshuang/datasets/models//QQP-all/', output_mode='classification', overwrite_cache=True, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, save_steps=1000000000000, seed=42, server_ip='', server_port='', task_name='qqp', tokenizer_name='', train_batch_size=16, warmup_steps=0, weight_decay=0.0)


In [4]:
# getting the data

df = read_one_dataset("quora", max_size=2000)
parsed_qs = list(zip(
    nlp.pipe(df.question1.tolist()), 
    nlp.pipe(df.question2.tolist())
))
labels = df.is_duplicate.tolist()
parsed_qs[0]

(What 's the best non touchscreen phone under 10K in India ?,
 What is currently the best phone under 10k in India ?)

# Vocabulary

In [5]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
print(', '.join(professions))

accountant, engineer, journalist, attorney, editor, artist, architect, actor, interpreter, analyst, intern, assistant, escort, organizer, economist, actress, administrator, investigator, agent, investor, secretary, author, entrepreneur, executive, advisor, auditor, educator, instructor, employee, adviser


In [6]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
professions = list(set(professions))
# professions
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [7]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:bert} {noun}?', noun=nouns)[:50]))
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']


-, [], ', ..., ", …, ­, un, /, super, over, ex, re, ​, [, under, anti, non, #, uber, auto, arch, extra, bio, Ex, Super, Under, ab, Un, mis, Uber, ultra, Re, e, in, , micro, Irish, sub, trans, outside, —, art, an, Air, news, American, dis, out, RE


In [8]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples


Evaluating: 100%|██████████| 125/125 [00:13<00:00,  9.36it/s]

Test cases:      1000
Fails (rate):    678 (67.8%)

Example fails:
1.0 ('Is Amber Harris an author?', 'Is Amber Harris a real author?')
----
1.0 ('Is Shannon Howard a journalist?', 'Is Shannon Howard a real journalist?')
----
1.0 ('Is Natalie Clark a candidate?', 'Is Natalie Clark an average candidate?')
----


In [13]:
test.results.passed

array([False, False, False,  True, False,  True, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True,  True, False, False, False, False,  True, False, False,
        True, False,  True,  True, False,  True,  True,  True, False,
       False, False, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
        True,  True, False,  True,  True,  True, False, False,  True,
        True, False,  True,  True,  True, False,  True, False, False,
       False,  True,  True, False,  True,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,  True,  True,  True, False, False, False,  True,
        True,  True,

In [10]:
test.data

[('Is Aaron Rodriguez a journalist?',
  'Is Aaron Rodriguez an English journalist?'),
 ('Is Elizabeth Sanchez an activist?',
  'Is Elizabeth Sanchez an elite activist?'),
 ('Is Tyler King an investigator?', 'Is Tyler King a honest investigator?'),
 ('Is Scott Martinez an intern?', 'Is Scott Martinez a good intern?'),
 ('Is Amber Harris an author?', 'Is Amber Harris a real author?'),
 ('Is Brian Moore a journalist?', 'Is Brian Moore an acomplished journalist?'),
 ('Is Jessica Sanders an interpreter?',
  'Is Jessica Sanders an amazing interpreter?'),
 ('Is Aaron Lewis an attorney?', 'Is Aaron Lewis an average attorney?'),
 ('Is Jose Cox a journalist?', 'Is Jose Cox a real journalist?'),
 ('Is Dylan Russell a photographer?',
  'Is Dylan Russell an incompetent photographer?'),
 ('Is Mary Wood an actress?', 'Is Mary Wood an excellent actress?'),
 ('Is Michael Peterson an adviser?', 'Is Michael Peterson an active adviser?'),
 ('Is Emily Phillips an escort?', 'Is Emily Phillips an official es

# NER

## Change same name, number, location in both

In [26]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both
    
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Predicting 297 examples


Evaluating: 100%|██████████| 38/38 [00:04<00:00,  9.48it/s]

Test cases:      27
Fails (rate):    4 (14.8%)

Example fails:
0.9 ('Did Hillary Clinton lie about Benghazi ?', 'Why did Hillary Clinton lie about Benghazi being attacked for a video instead of being honest to America ?')
0.0 ('Did Stephanie Ward lie about Benghazi ?', 'Why did Stephanie Ward lie about Benghazi being attacked for a video instead of being honest to America ?')
0.0 ('Did Nicole Martinez lie about Benghazi ?', 'Why did Nicole Martinez lie about Benghazi being attacked for a video instead of being honest to America ?')

----
1.0 ("Who runs Trump 's Twitter account ? Does Donald Trump write his own tweets ?", 'Does Donald Trump really write his own tweets ?')
0.0 ("Who runs Trump 's Twitter account ? Does Matthew Jones write his own tweets ?", 'Does Matthew Jones really write his own tweets ?')
0.1 ("Who runs Trump 's Twitter account ? Does Christopher Watson write his own tweets ?", 'Does Christopher Watson really write his own tweets ?')

----
0.0 ('Can you give me more u

In [27]:
test.data[0]

[('Why does people hate Taylor Swift ?', 'Why do people hate Taylor Swift ?'),
 ('Why does people hate Jennifer Garcia ?',
  'Why do people hate Jennifer Garcia ?'),
 ('Why does people hate Jessica Cox ?', 'Why do people hate Jessica Cox ?'),
 ('Why does people hate Ashley Carter ?',
  'Why do people hate Ashley Carter ?'),
 ('Why does people hate Sarah Morris ?', 'Why do people hate Sarah Morris ?'),
 ('Why does people hate Emily Parker ?', 'Why do people hate Emily Parker ?'),
 ('Why does people hate Amanda Powell ?',
  'Why do people hate Amanda Powell ?'),
 ('Why does people hate Elizabeth Bailey ?',
  'Why do people hate Elizabeth Bailey ?'),
 ('Why does people hate Melissa Sanders ?',
  'Why do people hate Melissa Sanders ?'),
 ('Why does people hate Stephanie Edwards ?',
  'Why do people hate Stephanie Edwards ?'),
 ('Why does people hate Nicole Diaz ?', 'Why do people hate Nicole Diaz ?')]

In [32]:
test.results.expect_results[1]

## Change name, loc, number in only one where orig prediction is duplicate

In [30]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 1332 examples


Evaluating: 100%|██████████| 167/167 [00:17<00:00,  9.40it/s]

Test cases:      82
After filtering: 43 (52.4%)
Fails (rate):    38 (88.4%)

Example fails:
0.9 ('Did Hillary Clinton lie about Benghazi ?', 'Why did Hillary Clinton lie about Benghazi being attacked for a video instead of being honest to America ?')
1.0 ('Did Brooke Clinton lie about Benghazi ?', 'Why did Hillary Clinton lie about Benghazi being attacked for a video instead of being honest to America ?')
1.0 ('Did Savannah Clinton lie about Benghazi ?', 'Why did Hillary Clinton lie about Benghazi being attacked for a video instead of being honest to America ?')

----
1.0 ('What will happen if Donald trump wins , and its effect on Indian students who are studying in US ?', 'What would be the scenario for Indian students after Donald trump becomes the President of US ?')
1.0 ('What will happen if Donald trump wins , and its effect on Indian students who are studying in US ?', 'What would be the scenario for Indian students after Brandon trump becomes the President of US ?')
1.0 ('What w

In [31]:
test.data[0]

[('Why should I not vote for Donald Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Dustin Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Jackson Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Stephen Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Logan Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Jose Trump ?',
  "Why should I not vote for Trump ? ( Someone asked me this today and I did n't know how to respond )"),
 ('Why should I not vote for Derek Trump ?',
  "Why should I not vote for Trump ? ( Someo

In [35]:
test.results.preds[0]

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

In [36]:
test.results.confs[0]

array([[6.2218524e-04, 9.9937773e-01],
       [9.8766202e-01, 1.2337957e-02],
       [1.5122294e-03, 9.9848771e-01],
       [1.8361963e-01, 8.1638026e-01],
       [9.8463297e-01, 1.5366935e-02],
       [8.3487844e-01, 1.6512157e-01],
       [2.0652901e-02, 9.7934705e-01],
       [9.7099119e-01, 2.9008875e-02],
       [9.4187868e-01, 5.8121365e-02],
       [3.8398802e-03, 9.9616027e-01],
       [9.9241483e-01, 7.5850831e-03]], dtype=float32)

In [34]:
test.results.expect_results[0]

array([-0.99937781,  1.        , -0.99848777, -0.81638037,  1.        ,
        1.        , -0.9793471 ,  1.        ,  1.        , -0.99616012,
        1.        ])

In [2]:

import checklist.viewer

In [3]:
print(checklist.viewer)

<module 'checklist.viewer' from '/home/wtshuang/sourcetree/checklist_ui/checklist/viewer/__init__.py'>


In [5]:
from checklist.viewer.viewer import TestSummarizer

In [7]:
summarize = {
            "tags": [],
            "name": "self.name",
            "result": {
                "nTested": 0,
                "nFailed": 0
            },
            "category": "Test"
        }

TestSummarizer(summarize, [])

TestSummarizer(stats={'nTested': 0, 'nFailed': 0}, summarizer={'tags': [], 'name': 'self.name', 'result': {'nT…

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [9]:
from viewer.viewer import TemplateEditor

TemplateEditor(
    sentences= {"Q1": "sentence1", "Q2": "sentence2"},
    suggest_fn=None,
    sources=[],
    allowed_sources=[],
    add_word_list_fn=None,
    get_testcase_fn=None,
    processor=nlp)

ModuleNotFoundError: No module named 'viewer'

In [2]:
def test():
    a=1
    b=2
    print(locals())

In [3]:
test()

{'a': 1, 'b': 2}


In [8]:
pos = ['good', 'bad', 'great']
def expect(x):
    return x in pos, locals()

In [9]:
expect("good")

(True, {'x': 'good'})

In [10]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def test():\n    a=1\n    b=2\n    print(locals()',
  'def test():\n    a=1\n    b=2\n    print(locals())',
  'test()',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, locals()",
  'expect("good")',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, globals()",
  'expect("good")',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, locals()",
  'expect("good")',
  'locals()'],
 '_oh': {5: (True, {'x': 'good'}), 7: (True, {...}), 9: (True, {'x': 'good'})},
 '_dh': ['/home/wtshuang/sourcetree/checklist_ui/notebooks'],
 'In': ['',
  'def test():\n    a=1\n    b=2\n    print(locals()',
  'def test():\n    a=1\n    b=2\n    print(locals())',
  'test()'

In [2]:
pos = ['good', 'bad', 'great']
def expect(x):
    return x in pos

In [4]:
import dill as pickle
pickle.dump(expect, open('./try_pickle.pkl', 'wb'), recurse=True)

In [3]:
import dill as pickle
expect = pickle.load(open('./try_pickle.pkl', 'rb'))

EOFError: Ran out of input